In [ ]:
# Importing Image class from PIL module 
from PIL import Image
import boto3

In [ ]:
#returns the different labels of objects that are found in the photo
def detect_labels(photo, bucket):

    client=boto3.client('rekognition')

    response = client.detect_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}},
        MaxLabels=15)

    return (response['Labels'])

In [ ]:
#checks if a person is detected in the photo and returns the objectbox values
def getPersonObjectBox(labels):

    for label in labels:
        if label['Name'] == "Person":
            for instance in label['Instances']:
                personObjectBox = {
                    "Type": "Person",
                    "width": instance['BoundingBox']['Width'],
                    "height": instance['BoundingBox']['Height'],
                    "top": instance['BoundingBox']['Top'],
                    "left": instance['BoundingBox']['Left'],
                }
                return personObjectBox
            else:
                return False

In [ ]:
def cropImage(left, upper, right, lower, index, path):

    #change path to use \ for windows compatibility

    path = path.replace('/', '\\')

    # Opens a image in RGB mode 
    im = Image.open(f'C:\ANPR_FOTOS\Alarm Picture\{path}')

    im_crop = im.crop((left, upper, right, lower))

    im_crop.save(f'C:\ANPR_FOTOS\Alarm Picture\Cropped\cropped{index}.jpg', 'JPEG')

In [ ]:
def calculateCoordinates(imageWidth, imageHeight, person):

    print(f'HIERE: {person}')

    #calculate the coördinates of the person objectBox 

    #calculate top/left (x,y) coördinate

    x1 = imageWidth * person['left']
    y1 = imageHeight * person['top']

    #calculate bottom/right (x,y) coördinate

    x2 = x1 + imageWidth * person['width']
    y2 = y1 + imageHeight * person['height']

    #add margins to get the windshield of the car left - 40%, right + 20%, top - 15% and bottom + 15%
    
    x1 = x1 - x1 * 0.4
    y1 = y1 - y1 * 0.15

    x2 = x2 + x2 * 0.2
    y2 = y2 + y2 * 0.15

    coordinates = {
        "x1": x1,
        "y1": y1,
        "x2": x2,
        "y2": y2,
    }

    return coordinates

In [ ]:
def main():
    # photo='Fotos-7-12-2020/20000101005741860_1_1SHP373_1_8.jpg'
    bucket='ai-essentials-anpr'

    #connect to the aws s3 bucket

    conn = client('s3')

    #loop over a list of all images in the s3 bucket and crop all windshields out of the photographs
    #enumerate is used to keep track of the index in the loop
    for index, key in enumerate(conn.list_objects(Bucket='ai-essentials-anpr')['Contents']):
        #key['Key'] is the name of a photo in the s3 bucket
        labels=detect_labels(key['Key'], bucket)

        person = getPersonObjectBox(labels)

        if(person):
            #image dimensions
            imageWidth = 1920
            imageHeight = 1144

            coordinates = calculateCoordinates(imageWidth, imageHeight, person)

            print(coordinates['x1'], coordinates['y1'])
            print(coordinates['x2'], coordinates['y2'])

            cropImage(coordinates['x1'], coordinates['y1'], coordinates['x2'], coordinates['y2'], index, key['Key'])
        else:
            print('No person detected')
        


if __name__ == "__main__":
    main()
